# Dataset Analysis

This notebook reproduces all figures from the paper using the datasets produced by running `make` in the scripts directory.
Brief description of the datasets:

* `naive_random.json` and `parameterised_random.json`:
Store parameters, features and performance metrics for instances generated using the naive and controlled generators.
In JSON format, each entry is an instance, with keys storing feature values.
The complete instance data is also generated and stored by the scripts (as compressed serialised numpy matrices) so they can be used as seed points for local search runs.

* `naive_search.json` and `parameterised_search.json`:
Store feature-space search records.
Each row records a search step.
The scripts record the full feature and performance metric set of the current instance found by local search every 100 steps.

* `naive_performance_search_*.json` and `parameterised_performance_search_*.json`:
Store performance-space search records.
File names indicate the performance metric used for search (e.g. `naive_performance_search_clp_primal_iterations.json`) stores the results of local search which aims to increase the number of iterations required by primal simplex to solve the instance.
Each row records a search step.
The scripts record the full feature and performance metric set of the current instance found by local search every 100 steps.

In [ ]:
import contextlib
import os
import json
from functools import partial
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import cm
import pandas as pd
import seaborn as sns
sns.set()
sns.set_style('white')

rename_columns = {
    'var_degree_max': 'Variable Degree Max',
    'cons_degree_max': 'Constraint Degree Max',
    'var_degree_min': 'Variable Degree Min',
    'cons_degree_min': 'Constraint Degree Min',
    'coefficient_density': 'Coefficient Density',
    'total_fractionality': 'Total Fractionality',
    'binding_constraints': 'Number of Binding Constraints',
    'rhs_mean': 'Constraint RHS Values Mean',
    'obj_mean': 'Objective Coefficients Mean',
    'clp_dual_iterations': 'Dual Simplex Iterations',
    'clp_barrier_iterations': 'Barrier Method Iterations',
    'clp_primal_iterations': 'Primal Simplex Iterations'
}

# Randomly generated datasets - rows are instances and columns are
# features/parameters/performance metrics.
df_naive_random = pd.read_json('data/naive_random.json').rename(columns=rename_columns)
df_naive_random['Generator'] = 'Naive'
df_param_random = pd.read_json('data/parameterised_random.json').rename(columns=rename_columns)
df_param_random['Generator'] = 'Controlled'

# Feature-space search datasets. Rows record the state of a search run at step N.
# Each dataframe contains step records for 100 runs.
df_naive_search = pd.read_json('data/naive_search.json').rename(columns=rename_columns)
df_param_search = pd.read_json('data/parameterised_search.json').rename(columns=rename_columns)

with contextlib.suppress(FileExistsError):
    os.mkdir('figures')

def save(name):
    plt.savefig(f'figures/{name}.pdf')

palette = sns.color_palette()

## Naive generator parameters

3000 instances generated, each with a new random seed and the following parameters.
Each 'sample' generates parameters from these distributions and runs the naive algorithm with those parameters.

```py
size_params = dict(variables=50, constraints=50)
rhs_params = dict(
    rhs_mean=uniform(low=-100.0, high=100.0),
    rhs_std=uniform(low=1.0, high=30.0))
objective_params = dict(
    obj_mean=uniform(low=-100.0, high=100.0),
    obj_std=uniform(low=1.0, high=10.0))
lhs_params = dict(
    density=uniform(low=0.0, high=1.0),
    pv=uniform(low=0.0, high=1.0),
    pc=uniform(low=0.0, high=1.0),
    coeff_loc=uniform(low=-2.0, high=2.0),
    coeff_scale=uniform(low=0.1, high=1.0))
```

## Controlled generator parameters

1000 instances generated, as above, using the controlled algorithm.
Generated more naive instances to ensure we had at least 1000 feasible, bounded instances as a comparison set.

```py
size_params = dict(variables=50, constraints=50)
beta_params = dict(
    basis_split=random_state.uniform(low=0.0, high=1.0))
alpha_params = dict(
    frac_violations=random_state.uniform(low=0.0, high=1.0),
    beta_param=random_state.lognormal(mean=-0.2, sigma=1.8),
    mean_primal=0,
    std_primal=1,
    mean_dual=0,
    std_dual=1)
lhs_params = dict(
    density=random_state.uniform(low=0.0, high=1.0),
    pv=random_state.uniform(low=0.0, high=1.0),
    pc=random_state.uniform(low=0.0, high=1.0),
    coeff_loc=random_state.uniform(low=-2.0, high=2.0),
    coeff_scale=random_state.uniform(low=0.1, high=1.0))
```

`df_naive_random` and `df_param_random` store features and performance metrics of randomly generated instances.
Each instance is a row generated using the parameters described above.

In [ ]:
print(f'Controlled generator samples: {df_param_random.shape[0]}')
print(f'Naive generator samples: {df_naive_random.shape[0]}')
print(f'Naive generator feasible bounded samples: {df_naive_random.solvable.sum()}')
print(f'Probability naive feasible and bounded: {df_naive_random.solvable.mean():.3f}')
df_param_random.head()

## Constraint Features

Plots below show features of the constraint matrix, which do not differ between generators, so we can just use the results in `df_param_random`.
Each point in the plots below represent an instance.

In [ ]:
sns.lmplot(
    data=df_param_random,
    x='Coefficient Density',
    y='Variable Degree Max',
    fit_reg=False)
save('feature-dist-density')

In [ ]:
ax = sns.lmplot(
    data=df_param_random,
    x='Constraint Degree Max',
    y='Variable Degree Max',
    fit_reg=False)
save('feature-dist-degree')

# Solution Features

Distribution of solution features differs between generators because the controlled generator explictly sets the solution.
These features are only relevant for feasible and bounded instances so the naive generator results are filtered.

In [ ]:
ax = sns.lmplot(
    data=pd.concat([
        df_param_random,
        df_naive_random[df_naive_random.solvable]]),
    x='Number of Binding Constraints',
    y='Total Fractionality',
    hue='Generator',
    hue_order=['Controlled', 'Naive'],
    palette=reversed(palette[0:2]),
    fit_reg=False)
ax.ax.set_ybound(lower=-1, upper=26)
save('feature-dist-relaxation')

In [ ]:
bins = [ 0.,  5., 10., 15., 20., 25., 30., 35., 40., 45., 50.]
df_param_random["Number of Binding Constraints"].hist(
    bins=bins, alpha=0.5)
df_naive_random[df_naive_random.solvable]["Number of Binding Constraints"].hist(
    bins=bins, alpha=0.5)
plt.gca().grid(False)
sns.despine()
plt.legend(["Controllable", "Naive"])
plt.xlabel("Number of Binding Constraints")
plt.ylabel("Number of Generated Instances")
save('feature-dist-binding')

# Objective/RHS Features

Plot below compares feasible and bounded instances from both generators by their positions in objective/rhs feature space.

In [ ]:
ax = sns.lmplot(
    data=pd.concat([
        df_param_random,
        df_naive_random[df_naive_random.solvable]]),
    x='Constraint RHS Values Mean',
    y='Objective Coefficients Mean',
    hue='Generator',
    hue_order=['Naive', 'Controlled'],
    palette=palette[0:2],
    fit_reg=False)
save('feature-dist-obj-rhs')

# Feasibility Phase Transition

The naive generator produces instances where the mean values of objective and rhs coefficients are uniformly distributed.
The plots below show feasibility and boundedness by position in this space.

In [ ]:
df_naive_random['Feasible and Bounded'] = df_naive_random['solvable'].apply(
    lambda v: 'Yes' if v else 'No')
sns.lmplot(
    data=df_naive_random,
    x='Constraint RHS Values Mean',
    y='Objective Coefficients Mean',
    hue='Feasible and Bounded',
    hue_order=['Yes', 'No'],
    palette=palette[2:],
    size=5, fit_reg=False, scatter_kws={'alpha': 0.6})
save('naive-transition-scatter')

In [ ]:
df = df_naive_random[df_naive_random.solvable]
sns.jointplot(
    x=df['Constraint RHS Values Mean'],
    y=df['Objective Coefficients Mean'],
    kind='kde', color='k', size=5, stat_func=None)
save('naive-transition-prob')

# Generated Instance Difficulty

Plots below indicate difficulty of instances for dual simplex in objective/rhs feature space.

In [ ]:
df = df_naive_random[df_naive_random.solvable].sample(1000)
print(f'Naive Generator. F+B samples: {df.shape[0]}')
fig, ax = plt.subplots()
sc = ax.scatter(
    x=df['Constraint RHS Values Mean'],
    y=df['Objective Coefficients Mean'],
    c=df['Dual Simplex Iterations'], cmap=cm.coolwarm)
ax.set_xbound([-125, 125])
ax.set_ybound([-125, 125])
ax.set_xlabel('Constraint RHS Values Mean')
ax.set_ylabel('Objective Coefficients Mean')
cb = plt.colorbar(sc)
cb.set_label('Dual Simplex Iterations')
sns.despine()
save('hardness-naive')

In [ ]:
df = df_param_random[df_param_random.solvable]
print(f'Parameterised Generator. Samples: {df.shape[0]}')
fig, ax = plt.subplots()
sc = ax.scatter(
    x=df['Constraint RHS Values Mean'],
    y=df['Objective Coefficients Mean'],
    c=df['Dual Simplex Iterations'], cmap=cm.coolwarm)
ax.set_xbound([-125, 125])
ax.set_ybound([-125, 125])
ax.set_xlabel('Constraint RHS Values Mean')
ax.set_ylabel('Objective Coefficients Mean')
cb = plt.colorbar(sc)
cb.set_label('Dual Simplex Iterations')
sns.despine()
save('hardness-controlled')

# Instance Difficulty

Shows instance difficulty achieved by the controllable generator as a direct result of setting the number of binding constraints in generated instances.

In [ ]:
df1 = df_naive_random[df_naive_random.solvable].sample(1000)
df2 = df_param_random[df_param_random.solvable]
df1['Generator'] = 'Naive'
df2['Generator'] = 'Controlled'
sns.lmplot(
    data=pd.concat([df1, df2]),
    x='Number of Binding Constraints',
    y='Dual Simplex Iterations',
    hue='Generator',
    hue_order=['Controlled', 'Naive'],
    palette=reversed(palette[0:2]),
    fit_reg=False,
    scatter_kws={'alpha': 0.4, 'linewidths': 0, 's': 60})
save('binding-constraints-hardness')

# Search

## Searching feature space

In all runs, the objective is simply to reach the target point (-100, 100) in the obj/rhs feature space (our missing region).
A start point is selected by randomly sampling 10 instances from the existing feasible-bounded instance set and choosing the closest point of those to the target.
At each local search step:
* generates a neighbour (using either naive or controllable method)
* accepts that neighbour if it is an improvement.

If the naive operator produces an infeasible or unbounded instance, it is rejected.
Search continues for 10000 of these steps and we repeat each search process 100 times.
The figure below shows state of play after 1000 steps.
Subsequent figure shows reduced distance to target after N steps.

In [ ]:
df1 = df_naive_random[df_naive_random.solvable].sample(1000).copy()
df1['Source'] = 'Naive Generator'
df2 = df_param_random[df_param_random.solvable].copy()
df2['Source'] = 'Controlled Generator'

step = 1000
df3 = df_naive_search[df_naive_search.solvable & (df_naive_search.step == step)].copy()
df3['Source'] = 'Naive Search'
df4 = df_param_search[df_param_search.solvable & (df_param_search.step == step)].copy()
df4['Source'] = 'Controlled Search'

g = sns.lmplot(
    data=pd.concat([df1, df2, df3, df4]),
    x='Constraint RHS Values Mean',
    y='Objective Coefficients Mean',
    hue='Source',
    fit_reg=False,
    hue_order=[
        'Naive Generator', 'Controlled Generator',
        'Naive Search', 'Controlled Search'],
    palette=palette
)
g.ax.set_xbound([-125, 125])
g.ax.set_ybound([-125, 125])
save('feature-search-target')

In [ ]:
def calc_objective(df):
    objective = (
        (df['Constraint RHS Values Mean'] + 100) ** 2 +
        (df['Objective Coefficients Mean'] - 100) ** 2) ** 0.5
    return pd.DataFrame(dict(step=df.step, objective=objective))

def compare_trajectory(labelled_series, ax, palette):
    ''' Expects series (step, objective) for comparison. '''
    assert len(labelled_series.items()) == 2
    for (label, data), color, linestyle in zip(labelled_series.items(), palette, ['--', '-']):
        df = data.groupby('step').objective.apply(
            lambda x: {
                'median': x.median(),
                'lower': x.quantile(0.25),
                'upper': x.quantile(0.75)}
            ).unstack(1).reset_index()
        ax.fill_between(df['step'], df['lower'], df['upper'], alpha=0.3, color=color)
        ax.plot(df['step'], df['median'], linestyle, color=color, label=label)
    ax.collections[0].set_hatch('xx')
    ax.set_xlabel('Search Step')
    ax.legend()
    return ax

In [ ]:
fig, ax = plt.subplots()
ax = compare_trajectory({
    'Naive': calc_objective(df_naive_search),
    'Controlled': calc_objective(df_param_search)
    }, ax, palette)
ax.semilogy()
ax.set_ylabel('Distance to Target')
sns.despine(fig)
save('feature-search-progression')

# Instance Difficulty

Distributions below compare the original naive and controlled generator datasets compared with the sets of instances found at the 10000th search step of each of the 100 feature space search processes by each operator.

In [ ]:
df1 = df_naive_random[df_naive_random.solvable].sample(1000).copy()
df2 = df_param_random[df_param_random.solvable].copy()

step = 10000
df3 = df_naive_search[df_naive_search.solvable & (df_naive_search.step == step)].copy()
df4 = df_param_search[df_param_search.solvable & (df_param_search.step == step)].copy()

key = 'Dual Simplex Iterations'
sns.boxplot(data=pd.DataFrame({
    'Naive\nGenerator': df1[key].reset_index(drop=True),
    'Controlled\nGenerator': df2[key].reset_index(drop=True),
    'Naive\nSearch': df3[key].reset_index(drop=True),
    'Controlled\nSearch': df4[key].reset_index(drop=True),
}), orient='h')
plt.xlabel('Dual Simplex Iterations')
plt.tight_layout()
sns.despine()
save('feature-search-hardness-dual-boxplot')

In [ ]:
key = 'Primal Simplex Iterations'
sns.boxplot(data=pd.DataFrame({
    'Naive\nGenerator': df1[key].reset_index(drop=True),
    'Controlled\nGenerator': df2[key].reset_index(drop=True),
    'Naive\nSearch': df3[key].reset_index(drop=True),
    'Controlled\nSearch': df4[key].reset_index(drop=True),
}), orient='h')
plt.xlabel('Primal Simplex Iterations')
plt.tight_layout()
sns.despine()
save('feature-search-hardness-primal-boxplot')

## Searching performance space

Operates in the same manner as feature space search, where the objective is to increase the number of iterations required by a target algorithm to solve the given feasible bounded instance.
The plots below show the search progression summarised over 100 runs with the same objective.

In [ ]:
def plot_for_field(ax, field):
    datasets = {
        'Naive': pd.read_json(
            f'data/naive_performance_search_{field}.json')[['step', field]].rename(
            columns={field: 'objective'}),
        'Controlled': pd.read_json(
            f'data/parameterised_performance_search_{field}.json')[['step', field]].rename(
            columns={field: 'objective'})}
    ax = compare_trajectory(datasets, ax, palette)
    return ax

In [ ]:
fig, ax = plt.subplots()
ax = plot_for_field(ax, 'clp_primal_iterations')
ax.set_ylabel('Primal Simplex Iterations')
sns.despine(fig)
plt.legend(loc=4)
save('search-primal')

In [ ]:
fig, ax = plt.subplots()
ax = plot_for_field(ax, 'clp_dual_iterations')
ax.set_ylabel('Dual Simplex Iterations')
sns.despine(fig)
save('search-dual')
plt.legend(loc=4)

In [ ]:
fig, ax = plt.subplots()
ax = plot_for_field(ax, 'clp_barrier_iterations')
ax.set_ylabel('Barrier Method Iterations')
sns.despine(fig)
save('search-barrier')
plt.legend(loc=4)

In [ ]:
cols = ['Primal Simplex Iterations', 'Dual Simplex Iterations', 'Barrier Method Iterations']

df1 = df_naive_random.loc[df_naive_random.solvable, cols]
df2 = df_param_random.loc[df_param_random.solvable, cols]

result1 = pd.DataFrame({
    ('Naive Generator', None, 'Q1'): df1.quantile(.25),
    ('Naive Generator', None, 'Q2'): df1.median(),
    ('Naive Generator', None, 'Q3'): df1.quantile(.75),
    ('Controllable Generator', None, 'Q1'): df2.quantile(.25),
    ('Controllable Generator', None, 'Q2'): df2.median(),
    ('Controllable Generator', None, 'Q3'): df2.quantile(.75),
}).transpose().unstack()
result1

In [ ]:
def _read(method, field):
    full_df = pd.read_json(f'data/{method}_performance_search_{field}.json')
    for step in [0, 200, 500, 1000]:
        df = full_df.loc[full_df.step == step, field]
        yield dict(
            Q1=df.quantile(.25), Q2=df.median(), Q3=df.quantile(.75),
            step=step, field=field, method=method)

import itertools

result = pd.DataFrame(list(itertools.chain(*(
    _read(method, field)
    for method, field in itertools.product(
        ['naive', 'parameterised'],
        ['clp_primal_iterations', 'clp_dual_iterations', 'clp_barrier_iterations']
    )))))
result['field'] = result['field'].map({
    'clp_dual_iterations': 'Dual Simplex Iterations',
    'clp_barrier_iterations': 'Barrier Method Iterations',
    'clp_primal_iterations': 'Primal Simplex Iterations'
})
result['method'] = result['method'].map({
    'naive': 'Naive Search',
    'parameterised': 'Controllable Search'
})
result = result.set_index(['field', 'method', 'step']).stack().unstack(0).unstack(2)
result = result[[
    'Primal Simplex Iterations',
    'Dual Simplex Iterations',
    'Barrier Method Iterations',
    ]]
result = pd.concat([result, result1]).sort_index()
result = (
    result
    .rename(columns=lambda col: col.replace(' Iterations', ''))
    .astype('int')
)
print(result.to_latex())
result

In [ ]:
# TODO add random dist values to each of these.

steps = [0, 100, 1000, 10000]

def _read():
    full_df = calc_objective(df_param_search)
    for step in steps:
        df = full_df.loc[full_df.step == step, 'objective']
        yield dict(method='Controllable Search', min=df.min(), median=df.median(), step=step)
    full_df = calc_objective(df_naive_search)
    for step in steps:
        df = full_df.loc[full_df.step == step, 'objective']
        yield dict(method='Naive Search', min=df.min(), median=df.median(), step=step)

result = pd.DataFrame(list(_read())).set_index(['method', 'step'])
print(result.round(3).to_latex())

# Design Solution Correctness

Generates an additional figure which shows whether the design solution features were produced correctly by the controlled generator.
The controlled generator attempts to set the number of nonzeros/binding constraints in the LP solutions.
This value may differ if the generated constraints lead to multiple solutions for the generated instance.
Figure below uses a rolling average to estimate probability that the solution found by simplex differs from the design solution, as a function of constraint matrix density (which affects likelihood of degeneracy).

In [ ]:
with open('data/parameterised_random.json') as infile:
    data = json.load(infile)

df = pd.DataFrame([
    {
        'basis_split': instance['params']['beta_params']['basis_split'],
        'binding_constraints': instance['binding_constraints'],
        'nonzeros': instance['nonzeros'],
    }
    for instance in data
])

solution_different = (df.binding_constraints - (df.basis_split * 50).round()).abs() > 0
(
    pd.DataFrame({
    'different': solution_different,
    'nonzeros': df.nonzeros
    }).groupby('nonzeros').different.mean().sort_index()
    .rolling(150, center=True).mean().dropna()
).plot()
plt.xlabel('Number of Nonzeros')
plt.ylabel('Pr (Solution Different)')
plt.xlim([150, 2300])
sns.despine()
save('pr-solution-different')